<a href="https://colab.research.google.com/github/rickymaulanafajri/SPA/blob/main/SPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 74.2 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 48.5 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.6 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.8/887.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.4 MB/s eta 0:00:00


In [ ]:
!pip install cdlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 82.2 MB/s eta 0:00:00


In [ ]:
from __future__ import division
from __future__ import print_function

import os
import argparse
import json
import codecs
from timeit import default_timer as timer
from copy import deepcopy

import random
import numpy as np

from sklearn import cluster
import sklearn.metrics as metrics

#import dgl
import networkx as nx
from networkx.algorithms.link_analysis.pagerank_alg import pagerank
from networkx.algorithms.community.quality import modularity
from networkx.utils.mapped_queue import MappedQueue

import torch
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GATConv
from torch_geometric.datasets import Planetoid, CoraFull, Coauthor,WikiCS,GitHub,HeterophilousGraphDataset
#from ogb.nodeproppred import PygNodePropPredDataset

from cdlib import algorithms
import numpy as np

Note: to be able to use all crisp methods, you need to install some additional packages:  {'infomap', 'leidenalg', 'wurlitzer', 'graph_tool', 'bayanpy'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'leidenalg', 'infomap', 'wurlitzer'}


In [ ]:
class ActiveLearning:
    """
    An active learning framework that...
    * queries from an oracle;
    * updates its known set,
    * trains the GNN model, and
    * evaluate the Macro F-1 score.
    """

    def __init__(self, data, model, seed, args):
        self.round = 0
        self.data = data
        self.model = model
        self.seed = seed
        self.args = args
        self.retrain = args.retrain
        self.clf = None
        self.aggregated = None
        self.num_centers = args.num_centers
        self.num_parts = -1

    def query(self, b):
        pass

    def update(self, train_mask):
        self.data.train_mask = train_mask
        self.round += 1

    def train(self):
        if self.retrain:
            self.clf = deepcopy(self.model).to(self.args.device)
        else:
            self.clf = self.model.to(self.args.device)
        optimizer = optim.Adam(
            self.clf.parameters(), lr=self.args.lr,
            weight_decay=self.args.weight_decay)
        for epoch in range(self.args.epochs):
            self.clf.train()
            optimizer.zero_grad()
            out = self.clf(self.data.x, self.data.adj_t)
            true = self.data.y
            if len(true.shape) > 1:
                true = true.squeeze(1)
            loss = F.cross_entropy(
                out[self.data.train_mask],
                true[self.data.train_mask])
            if self.args.verbose == 2:
                print('Epoch {:03d}: Training loss: {:.4f}'.format(epoch, loss))
            loss.backward()
            optimizer.step()

    def evaluate(self):
        self.clf.eval()
        logits = self.clf(self.data.x, self.data.adj_t)
        y_pred = logits.max(1)[1].cpu()
        y_true = self.data.y.cpu()
        f1 = metrics.f1_score(y_true, y_pred, average='macro')
        acc = metrics.f1_score(y_true, y_pred, average='micro')
        if self.args.verbose == 2:
            print('Macro-f1 score: {:.4f}'.format(f1))
            print('Micro-f1 score: {:.4f}'.format(acc))
        return f1, acc

    def get_node_representation(self, rep='aggregation', encoder='gcn'):

        if rep == 'aggregation':
            if self.aggregated is None:
                feat_dim = self.data.x.size(1)
                if encoder == 'sage':
                    conv = SAGEConv(feat_dim, feat_dim, bias=False)
                    conv.lin_l.weight = torch.nn.Parameter(torch.eye(feat_dim))
                    conv.lin_r.weight = torch.nn.Parameter(torch.eye(feat_dim))
                else:
                    conv = GCNConv(feat_dim, feat_dim, cached=True, bias=False)
                    conv.lin.weight = torch.nn.Parameter(torch.eye(feat_dim))
                conv.to(self.args.device)
                with torch.no_grad():
                    self.aggregated = conv(self.data.x, self.data.adj_t)
                    self.aggregated = conv(self.aggregated, self.data.adj_t)
            return self.aggregated

        elif rep == 'embedding':
            with torch.no_grad():
                embed = self.clf.embed(self.data.x, self.data.adj_t)
            return embed

        else:
            return self.data.x

    def split_cluster(self, b, partitions, x_embed=None, method='default'):

        if method == 'inertia':
            part_size = []
            for i in range(self.num_parts):
                part_id = np.where(partitions == i)[0]
                x = x_embed[part_id]
                kmeans = Cluster(n_clusters=1, n_dim=x_embed.shape[1], seed=self.seed, device=self.args.device)
                kmeans.train(x.cpu())
                inertia = kmeans.get_inertia()
                part_size.append(inertia)

            part_size = np.rint(b * np.array(part_size) / sum(part_size)).astype(int)
            part_size = np.maximum(self.num_centers, part_size)
            i = 0
            while part_size.sum() - b != 0:
                if part_size.sum() - b > 0:
                    i = self.num_parts - 1 if i <= 0 else i
                    while part_size[i] <= 1:
                        i -= 1
                    part_size[i] -= 1
                    i -= 1
                else:
                    i = 0 if i >= self.num_parts else i
                    part_size[i] += 1
                    i += 1

        elif method == 'size':
            part_size = []
            for i in range(self.num_parts):
                part_size.append(len(np.where(partitions == i)[0]))
            part_size = np.rint(b * np.array(part_size) / sum(part_size)).astype(int)
            part_size = np.maximum(self.num_centers, part_size)
            i = 0
            while part_size.sum() - b != 0:
                if part_size.sum() - b > 0:
                    i = self.num_parts - 1 if i <= 0 else i
                    while part_size[i] <= 1:
                        i -= 1
                    part_size[i] -= 1
                    i -= 1
                else:
                    i = 0 if i >= self.num_parts else i
                    part_size[i] += 1
                    i += 1

        else:
            part_size = [b // self.num_parts for _ in range(self.num_parts)]
            for i in range(b % self.num_parts):
                part_size[i] += 1

        return part_size

    def __str__(self):
        return "Active Learning Agent (uninitialized)"


class Random(ActiveLearning):
    """
    Random:
    The Random Sampling method chooses nodes uniformly at random,
    similarly as the commonly used semi-supervised learning experiment setting for GCN.
    """

    def __init__(self, data, model, seed, args):
        super(Random, self).__init__(data, model, seed, args)

    def query(self, b):
        indice = np.random.choice(
            np.where(self.data.train_mask == 0)[0], b, replace=False
        )
        return torch.tensor(indice)

    def __str__(self):
        return "Random"



class Uncertainty(ActiveLearning):
    """
    Uncertainty:
    The Uncertainty method chooses the nodes with maximum entropy on the predicted class distribution.
    """

    def __init__(self, data, model, seed, args):
        super(Uncertainty, self).__init__(data, model, seed, args)

    def query(self, b):
        logits = self.clf(self.data.x, self.data.adj_t)
        entropy = -torch.sum(F.softmax(logits, dim=1) * F.log_softmax(logits, dim=1), dim=1)
        entropy[np.where(self.data.train_mask != 0)[0]] = 0
        _, indices = torch.topk(entropy, k=b)
        return indices

    def __str__(self):
        return "Uncertainty"


#our proposed method
class SCANPageRankOptimized(ActiveLearning):
    """
    Sampling based on representativeness within communities using a dynamic approach with the SCAN algorithm for community detection. PageRank centrality, with the option for Personalized PageRank, is used as the metric for representativeness.
    """

    def __init__(self, data, model, seed, args, personalization=None, damping=0.95, max_iter=400, tol=1.0e-6, epsilon=0.5, mu=3):
        super(SCANPageRankOptimized, self).__init__(data, model, seed, args)
        self.personalization = personalization
        self.damping = damping
        self.max_iter = max_iter
        self.tol = tol
        self.epsilon = epsilon
        self.mu = mu
        self.previous_G = None

    def compute_pagerank(self, G):
        # Check if PageRank was already computed and if the graph has not changed
        if not hasattr(self, 'pagerank_scores') or self.previous_G != G:
            self.pagerank_scores = nx.pagerank(G, alpha=self.damping, personalization=self.personalization, max_iter=self.max_iter, tol=self.tol)
            self.previous_G = G
        return self.pagerank_scores

    def query(self, b):
        # Convert adjacency matrix to graph
        adj_matrix = self.data.adj_t.to_dense().numpy()
        G = nx.from_numpy_array(adj_matrix)

        # Calculate PageRank for each node
        pagerank_scores = self.compute_pagerank(G)

        selected_indices = []
        remaining_budget = b

        while remaining_budget > 0:
            # Apply the SCAN algorithm for community detection
            partition = algorithms.scan(G, epsilon=self.epsilon, mu=self.mu)
            communities = partition.communities

            # Select nodes from each community based on PageRank scores
            for community in communities:
                if not community:
                    continue
                community_nodes = sorted([(node, pagerank_scores[node]) for node in community], key=lambda x: x[1], reverse=True)
                num_samples = max(1, remaining_budget // len(communities))  # Ensure at least one sample per community
                selected_indices.extend([node for node, _ in community_nodes[:num_samples]])
                remaining_budget -= num_samples

            # Break if no more selections are needed
            if remaining_budget <= 0:
                break

        # Handle case where fewer nodes are selected than requested
        if len(selected_indices) < b:
            all_nodes_sorted = sorted(pagerank_scores.items(), key=lambda x: x[1], reverse=True)
            remaining_indices = [node for node, _ in all_nodes_sorted if node not in selected_indices]
            selected_indices.extend(remaining_indices[:b - len(selected_indices)])

        # Convert to tensor
        selected_indices = torch.tensor(selected_indices, dtype=torch.long)

        return selected_indices[:b]




In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout=0.5, batchnorm=False, activation="relu"):
        super(GCN, self).__init__()
        assert activation in ["relu", "elu"]
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        if batchnorm:
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.num_layers = num_layers
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels, cached=True))
            if batchnorm:
                self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))
        self.dropout = dropout
        self.activation = getattr(F, activation)

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        x = self.embed(x, adj_t)
        x = self.convs[-1](x, adj_t)
        return x

    def embed(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            if len(self.bns) > 0:
                x = self.bns[i](x)
            x = self.activation(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        return x

class SAGE(torch.nn.Module):
    """
    GraphSAGE
    """
    def __init__(self, in_channels, hidden_channels, out_channels,
                 num_layers=16, dropout=0.5, batchnorm=False, activation="relu"):
        super(SAGE, self).__init__()

        assert activation in ["relu", "elu"]

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        if batchnorm:
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))

        self.num_layers = num_layers
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            if batchnorm:
                self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout
        self.activation = getattr(F, activation)

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        x = self.embed(x, adj_t)
        x = self.convs[-1](x, adj_t)
        return x

    def embed(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            if len(self.bns) > 0:
                x = self.bns[i](x)
            x = self.activation(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        return x




In [ ]:
def run(data, args):

    gnn = args.model
    baseline = args.baselines
    budget = int(args.budget)
    seed = int(args.seed)

    # Choose model
    model_args = {
        "in_channels": data.num_features,
        "out_channels": data.num_classes,
        "hidden_channels": args.hidden,
        "num_layers": args.num_layers,
        "dropout": args.dropout,
        "activation": args.activation,
        "batchnorm": args.batchnorm
    }

    # Initialize models
    if gnn == "gcn":
        model = GCN(**model_args)
    elif gnn == "sage":
        model = SAGE(**model_args)
    else:
        raise NotImplemented

    model = model.to(args.device)

    # Methods
    if baseline == "random":
        agent = Random(data, model, seed, args)
    elif baseline == "uncertainty":
        agent = Uncertainty(data, model, seed, args)
    elif baseline == "scanpage":
        agent = SCANPageRankOptimized(data,model,seed,args)
       # Initialization
    training_mask = np.zeros(data.num_nodes, dtype=bool)
    initial_mask = np.arange(data.num_nodes)
    np.random.shuffle(initial_mask)
    init = args.init
    if baseline in ['uncertainty']:
        init = budget // 3
    training_mask[initial_mask[:init]] = True

    training_mask = torch.tensor(training_mask)
    agent.update(training_mask)
    agent.train()

    if args.verbose > 0:
        print('Round {:03d}: Labelled: {:d}, Prediction macro-f1 score {:.4f}'
              .format(0, init, agent.evaluate()))

    # Query
    start = timer()
    indices = agent.query(budget - init)
    end = timer()
    print('Total Query Runtime [s]:', end - start)

    # Update
    training_mask[indices] = True
    agent.update(training_mask)

    # Training
    agent.train()

    # Evaluate
    f1, acc = agent.evaluate()
    labelled = len(np.where(agent.data.train_mask != 0)[0])

    if args.verbose > 0:
        print('Round {:03d}: # Labelled nodes: {:d}, Prediction macro-f1 score {:.4f}'
              .format(rd, labelled, f1))
    else:
        pass

    return f1, acc, indices, agent

In [ ]:
def test(data, gnns, budgets, baselines, seed=0):

    # Set seeds
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    # Training settings
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--verbose", type=int, default=0, help="Verbose: 0, 1 or 2")
    parser.add_argument(
        "--device", default=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    # General configs
    parser.add_argument(
        "--baselines", type=str, default=baselines)
    parser.add_argument(
        "--model", default=gnns)
    parser.add_argument(
        "--partition", type=str, default='greedy')

    # Active Learning parameters
    parser.add_argument(
        "--budget", type=int, default=budgets,
        help="Number of rounds to run the agent.")
    parser.add_argument(
        "--retrain", type=bool, default=True)
    parser.add_argument(
        "--num_centers", type=int, default=1)
    parser.add_argument(
        "--representation", type=str, default='aggregation')
    parser.add_argument(
        "--compensation", type=float, default=1.0)
    parser.add_argument(
        "--init", type=float, default=0, help="Number of initially labelled nodes.")
    parser.add_argument(
        "--epochs", type=int, default=300, help="Number of epochs to train.")
    parser.add_argument(
        "--steps", type=int, default=4, help="Number of steps of random walk.")

    # GNN parameters
    parser.add_argument(
        "--seed", type=int, default=seed, help="Number of random seeds.")
    parser.add_argument(
        "--lr", type=float, default=0.01, help="Initial learning rate.")
    parser.add_argument(
        "--weight_decay", type=float, default=5e-4,
        help="Weight decay (L2 loss on parameters).")
    parser.add_argument(
        "--hidden", type=int, default=16, help="Number of hidden units.")
    parser.add_argument(
        "--num_layers", type=int, default=2, help="Number of layers.")
    parser.add_argument(
        "--dropout", type=float, default=0,
        help="Dropout rate (1 - keep probability).")
    parser.add_argument(
        "--batchnorm", type=bool, default=False,
        help="Perform batch normalization")
    parser.add_argument(
        "--activation", default="relu")

    # GAT hyper-parameters
    parser.add_argument(
        "--num_heads", type=int, default=8, help="Number of heads.")

    args, _ = parser.parse_known_args()

    f1_all, acc_all, queried, agent = run(data, args)

    agg = agent.get_node_representation('aggregation', 'gcn').cpu().numpy()
    agg_distance = {}
    train_idx = list(queried.numpy())
    test_idx = list(range(data.num_nodes))
    train_agg = agg[train_idx]

    for i in test_idx:
        distance_tmp = train_agg - agg[i]
        agg_distance[int(i)] = float(min(np.linalg.norm(distance_tmp, axis=1)))

    group_num = 10
    sort_res = list(
        map(lambda x: x[0], sorted(agg_distance.items(), key=lambda x: x[1])))
    node_num_group = len(sort_res) // group_num
    res = [
        sort_res[i:i + node_num_group + 1]
        for i in range(0, len(sort_res), node_num_group + 1)
    ]

    acc_list = []
    f1_list = []
    agent.clf.eval()
    logits = agent.clf(agent.data.x, agent.data.adj_t)
    y_pred = logits.max(1)[1].cpu()
    y_true = agent.data.y.cpu()
    for test_set in res:
        acc = metrics.accuracy_score(y_true[test_set], y_pred[test_set])
        f1 = metrics.f1_score(y_true[test_set], y_pred[test_set], average='macro')
        acc_list.append(acc)
        f1_list.append(f1)
    print(f"Macro F1 {np.mean(f1_list)}")
    #print(f"Accuracy {np.mean(acc_list)}")


In [ ]:
name = 'citeseer'

path = os.path.join("data", name)
#cora, citeseer, pubmed using this Planetoid dataset class
dataset = Planetoid(root=path, name=name, transform=T.ToSparseTensor())

#use other dataset here

data = dataset[0]
data.max_part = 14
data.num_classes = dataset.num_classes
data.params = {'age': [0.35, 0.35, 0.3]}

print(data.num_nodes)
print(data.num_edges)
print(data.num_classes)
print(data.x.shape[1])



Processing...


3327
9104
6
3703


Done!


In [ ]:
#change this part with 'gcn', or 'sage' for different graph architecture
test(data,'gcn',40,'scanpage',seed=43)


Total Query Runtime [s]: 0.19624310200003947
Macro F1 0.59155245280681
